## OpenAI pdf chatbot

- PDF 챗봇은 보통 다음과 같은 순서로 만들어진다.
1. PDF 파일 로드
2. 문서 텍스트 분할
3. 벡터 임베딩
4. 벡터 DB 생성 및 임베딩 값 저장

   ----------------- 챗봇 구현 사전 단계 --------------------

5. 사용자 질의를 임베딩 후 vector DB에서 검색 (Retriever 사용)
6. QA Chain을 통해 LLM에서 답변을 생성받아 사용자에게 제공
   - 프롬프트 엔지니어링
   - llm 모델 정의
   - (프롬포트-llm-출력) chain 생성

In [ ]:
import os
# API 키 설정
os.environ["OPENAI_API_KEY"] = 'your-api-key'

In [79]:
#import
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQAWithSourcesChain
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [80]:
# PDF 파일 로드
loader = PyPDFDirectoryLoader("data/")
docs = loader.load()

In [81]:
# 문서 텍스트 분할
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50,
    length_function = len,
    separators=["\n\n", "\n", " ",""],
)

text_chunks = [] 
for doc in docs:
    chunks = text_splitter.split_text(doc.page_content)
    text_chunks.extend([Document(page_content=chunk) for chunk in chunks])

In [82]:
# FAISS 데이터베이스 생성 및 벡터 임베딩
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
vectorstore = FAISS.from_documents(
    documents=text_chunks,
    embedding=embeddings
)

In [83]:
# Retriever 정의
retriever = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={'k': 6, 'lambda_mult': 0.25}
    )

In [84]:
# 프롬프트 생성
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Answer in Korean.

#Question: 
{question} 
#Context: 
{context} 

#Answer:"""
)

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)  

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [85]:
question = "기후 공시가 뭐야?"
response = chain.invoke(question)
print(response)

기후 공시는 기업이 기후변화 관련 정보를 공개하는 것을 말합니다.
